In [ ]:
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import keras
from sklearn.dummy import DummyClassifier
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from numpy import array
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from pandas.plotting import lag_plot

Using TensorFlow backend.


In [ ]:
#Function for loading data!

def load_data():
    
    df = pickle.load(open('df_down_sampled.p','rb'))
    df = df.drop(['resp_1', 'resp_2','resp_3','resp_4'], axis = 1)
    df = df.sort_values(by = 'ts_id')
    df['y'] = 0
    mask = df.resp > 0
    df.loc[mask,'y'] = 1    
    
    return df

In [ ]:
#Function for performing pca-transformation!

def perform_pca_transformation(X_train, X_test, number_of_components):
    
    col_list = []

    for x in range(number_of_components):
        
        col_list.append('pca_feature_' + str(x))
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    pca = PCA(n_components = number_of_components)
    pca.fit(X_train_scaled)
    
    X_train_pca = pca.transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    X_train_pca = pd.DataFrame(data = X_train_pca, columns = col_list)
    X_test_pca = pd.DataFrame(data = X_test_pca, columns = col_list)
    
    return (X_train_pca, X_test_pca)

In [ ]:
#Function for splitting data into train/test set!

def train_test_split(test_share, data):
    
    #Split data into initial train/test
    
    train_share = 1 - test_share    
    train_size = int(len(data) * train_share)
    train_set = data[0:train_size]
    test_set = data[train_size:len(data)]    
    
    
    return (train_set, test_set)

In [345]:
#Load data!

df = load_data()


In [346]:
#Fill in missing values with mean for now!

df = df.fillna(df.mean())


In [347]:
#Split into train/test!

train_set, test_set = train_test_split(test_share = 0.9, data = df)

In [348]:
X_train = train_set.drop(['date','weight','y','ts_id', 'resp'], axis = 1)
X_test = test_set.drop(['date','weight','y','ts_id', 'resp'], axis = 1)

y_train = train_set.y
y_test = test_set.y

In [349]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

In [335]:
(X_train_pca, X_test_pca) = perform_pca_transformation(X_train, X_test, 20)

C:\Users\mathias.buxhoeveden\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\mathias.buxhoeveden\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\mathias.buxhoeveden\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if sys.path[0] == '':


In [336]:
model = XGBClassifier()
model.fit(X_train_pca, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

In [337]:
y_pred_train = model.predict_proba(X_train_pca)[:,1]

In [338]:
y_pred_test = model.predict_proba(X_test_pca)[:,1]

In [339]:
train_roc = roc_auc_score(y_train, y_pred_train)

In [340]:
test_roc = roc_auc_score(y_test, y_pred_test)

In [341]:
print("The train roc is: {}, the test roc is: {}".format(train_roc, test_roc))

The train roc is: 0.7081934935725717, the test roc is: 0.5136428801937363


In [329]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

In [330]:
y_pred_train = model.predict_proba(X_train)[:,1]

In [331]:
train_roc = roc_auc_score(y_test, y_pred_test)

In [332]:
y_pred_test = model.predict_proba(X_test)[:,1]

In [333]:
test_roc = roc_auc_score(y_test, y_pred_test)

In [334]:
print("The train roc is: {}, the test roc is: {}".format(train_roc, test_roc))

The train roc is: 0.5119729302914438, the test roc is: 0.513748571503123
